In [14]:
import pyworld as pw
import librosa
import numpy as np
import os
import math
import torch
from pymcd.mcd import Calculate_MCD
import audio as Audio
# wav2vecSER Model
from speechbrain.inference.interfaces import foreign_class

In [27]:
ground_truth_path = "/home/johnny9696/Desktop/DATA/dailytalk_resample/data"
#model_output_path = "/mnt/hdd_storage/di_fit_tts_output/output/dailytalk"
#model_output_path = "/mnt/hdd_storage/di_fit_tts_output/output/Fitted_v19_flowv2_MPE"
model_output_path = "/mnt/hdd_storage/di_fit_tts_output/output/fs2"
sampling_rate = 22050
hop_length = 256
d_list = os.listdir(model_output_path)
print(d_list)

['d1653', 'd122', 'd2210', 'd1403', 'd79', 'd2077', 'd851', 'd2046', 'd1780', 'd520', 'd314', 'd1514', 'd2347', 'd1510', 'd1406', 'd213', 'd1248', 'd970', 'd1363', 'd295', 'd534', 'd2447', 'd115', 'd1542', 'd1774', 'd865', 'd1394', 'd1533', 'd236', 'd597', 'd1758', 'd1815', 'd1360', 'd1108', 'd2016', 'd794', 'd2400', 'd1075', 'd1927', 'd1856', 'd339', 'd2184', 'd283', 'd925', 'd1804', 'd954', 'd299', 'd1282', 'd1553', 'd2263', 'd1918', 'd1017', 'd87', 'd1180', 'd470', 'd2309', 'd872', 'd1700', 'd440', 'd1262', 'd1289', 'd1311', 'd2216', 'd2052', 'd1529', 'd2294', 'd47', 'd1434', 'd2511', 'd280', 'd1404', 'd279', 'd2459', 'd382', 'd1190', 'd2183', 'd718', 'd404', 'd2497', 'd777', 'd1233', 'd1632', 'd1222', 'd2515', 'd707', 'd172', 'd24', 'd40', 'd2', 'd1873', 'd2489', 'd2236', 'd1516', 'd2015', 'd168', 'd1937', 'd189', 'd222', 'd766', 'd800', 'd1645', 'd3', 'd466', 'd1905', 'd2524', 'd20', 'd855', 'd2382', 'd2131', 'd622', 'd1597', 'd2149', 'd2304', 'd1865', 'd1750', 'd2266', 'd1257', '

In [28]:
def get_pitch(wav, sampling_rate = 22050, hop_length = 256):
    pitch, t = pw.dio(
        wav.astype(np.float64),
        sampling_rate,
        frame_period=hop_length / sampling_rate * 1000,
    )
    pitch = pw.stonemask(wav.astype(np.float64), pitch, t, sampling_rate)
    return pitch

In [29]:
def get_energy(wav):
# Compute mel-scale spectrogram and energy
        STFT = Audio.stft.TacotronSTFT(
            1024,
            256,
            1024,
            80,
            22050,
            0,
            8000,
        )
        _, energy = Audio.tools.get_mel_from_wav(wav, STFT)
        return energy
        

In [30]:
count = 0
wav_rmse = 0
f0_rmse = 0
for i in d_list:
    t_output_path = os.path.join(model_output_path, i)
    t_gt_path = os.path.join(ground_truth_path, i[1:])
    w_list = os.listdir(t_output_path)
    for wav_name in w_list:
        if wav_name[-3:] !="wav":
            continue
        gt_wav, sr = librosa.load(os.path.join(t_gt_path, wav_name))
        gen_wav, sr = librosa.load(os.path.join(t_output_path, wav_name))
        gt_p = get_pitch(gt_wav)
        gen_p = get_pitch(gen_wav)
        if len(gt_p) > len(gen_p):
            gap = len(gt_p) - len(gen_p)
            gt_p = gt_p[gap//2:-gap//2]
            gen_p = gen_p[:len(gt_p)]
        else:
            gap = len(gen_p) - len(gt_p)
            gen_p = gen_p[gap//2:-gap//2]
            gt_p = gt_p[:len(gt_p)]
        try:
            gt_log = np.nan_to_num(np.log(gt_p))
            gen_log = np.nan_to_num(np.log(gen_p))
            error = abs(gt_log-gen_log)
            non_inf_counter = 0
            rmse = 0
            for data in error:
                if data != np.inf and data != -np.inf and data>= -10 and data <=10:
                    non_inf_counter += 1
                    rmse += data ** 2
            rmse = np.sqrt(rmse/non_inf_counter)
            f0_rmse += rmse
            count += 1
            if count%100 == 0:
                print(count, f0_rmse/count)
        except:
            pass

/tmp/ipykernel_93762/3240988841.py:24: RuntimeWarning: divide by zero encountered in log
  gt_log = np.nan_to_num(np.log(gt_p))
/tmp/ipykernel_93762/3240988841.py:25: RuntimeWarning: divide by zero encountered in log
  gen_log = np.nan_to_num(np.log(gen_p))


100 0.2622553493807052
200 0.27224669272898266
300 0.2723190405592484
400 0.27082216030843465
500 0.27317950056246637
600 0.27314485067342226
700 0.27155581829409065
800 0.2713297779482404
900 0.27378217248970205
1000 0.27221664524959227
1100 0.2774513412521834
1200 0.2761424221721954
1300 0.2754698341593281
1400 0.2741265395185927
1500 0.2733489440427654
1600 0.27317135941765913
1700 0.273377430935769
1800 0.27194488623060187


In [31]:
print(f0_rmse/count)

0.2725826021264256


In [32]:
count = 0
e_rmse = 0
for i in d_list:
    t_output_path = os.path.join(model_output_path, i)
    t_gt_path = os.path.join(ground_truth_path, i[1:])
    w_list = os.listdir(t_output_path)
    for wav_name in w_list:
        if wav_name[-3:] !="wav":
            continue
        gt_wav, sr = librosa.load(os.path.join(t_gt_path, wav_name))
        gen_wav, sr = librosa.load(os.path.join(t_output_path, wav_name))
        gt_p = get_energy(gt_wav)
        gen_p = get_energy(gen_wav)
        if len(gt_p) > len(gen_p):
            gap = len(gt_p) - len(gen_p)
            gt_p = gt_p[gap//2:-gap//2]
            gen_p = gen_p[:len(gt_p)]
        else:
            gap = len(gen_p) - len(gt_p)
            gen_p = gen_p[gap//2:-gap//2]
            gt_p = gt_p[:len(gt_p)]
        try:
            gt_log = np.nan_to_num(np.log(gt_p))
            gen_log = np.nan_to_num(np.log(gen_p))
            error = abs(gt_log-gen_log)
            non_inf_counter = 0
            rmse = 0
            for data in error:
                if data != np.inf and data != -np.inf and data>= -10 and data <=10:
                    non_inf_counter += 1
                    rmse += data ** 2
            rmse = np.sqrt(rmse/non_inf_counter)
            e_rmse += rmse
            count += 1
            if count%100 == 0:
                print(count, e_rmse/count)

        except:
            pass

100 2.236316002398535
200 2.1933097876569785
300 2.1842831585559135
400 2.2203354207370136
500 2.2135415732170785
600 2.223354787546786
700 2.22831623330988
800 2.205511780173614
900 2.195113002306916
1000 2.1927496430898867
1100 2.2007643248146787
1200 2.217600462276618
1300 2.221420117738284
1400 2.218626925372667
1500 2.2223912286237177
1600 2.2403640266782157
1700 2.235490814385781
1800 2.228980665565161


In [33]:
print(e_rmse/count)

2.2236093952331655


In [22]:
audio_embedder = foreign_class(source="speechbrain/emotion-recognition-wav2vec2-IEMOCAP",
                                       pymodule_file="custom_interface.py", classname="CustomEncoderWav2vec2Classifier")

Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
speechbrain.lobes.models.huggingface_transformers.huggingface - Wav2Vec2Model is frozen.


In [34]:
w_count = 0
wav_rmse = 0
for i in d_list:
    t_output_path = os.path.join(model_output_path, i)
    t_gt_path = os.path.join(ground_truth_path, i[1:])
    w_list = os.listdir(t_output_path)
    for wav_name in w_list:
        if wav_name[-3:] !="wav":
            continue
        gt_wav, sr = librosa.load(os.path.join(t_gt_path, wav_name))
        gen_wav, sr = librosa.load(os.path.join(t_output_path, wav_name))
        gt_wav_e = torch.tensor(gt_wav)
        gt_wav_emb = audio_embedder.encode_batch(gt_wav_e)
        gt_wav_emb = np.array(gt_wav_emb.squeeze(0))
        gen_wav_e = torch.tensor(gen_wav)
        gen_wav_emb = audio_embedder.encode_batch(gen_wav_e)
        gen_wav_emb = np.array(gen_wav_emb.squeeze(0))
        error = abs(gt_wav_emb - gen_wav_emb)
        rmse = np.sqrt(np.mean(error**2))
        wav_rmse += rmse
        w_count += 1
        if w_count%100 == 0:
            print(w_count, wav_rmse/w_count)

100 0.8347542335093021
200 0.860781531855464
300 0.8707387032608191
400 0.8580506410822273
500 0.8438284749537707
600 0.8516770665720105
700 0.8474428770691156
800 0.8441312044765801
900 0.8468414485537343
1000 0.8549331518188119
1100 0.8564677710627968
1200 0.8577232097027202
1300 0.8560013628521791
1400 0.8576178757154516
1500 0.8574154142091671
1600 0.8542872145446018
1700 0.8575794335454703
1800 0.8576563020423055


In [35]:
print(wav_rmse/w_count)

0.8584494442032039


In [36]:
mcd_count = 0
mcd = 0
mcd_toolbox = Calculate_MCD(MCD_mode="dtw")
for i in d_list:
    t_output_path = os.path.join(model_output_path, i)
    t_gt_path = os.path.join(ground_truth_path, i[1:])
    w_list = os.listdir(t_output_path)
    for wav_name in w_list:
        if wav_name[-3:] !="wav":
            continue
        mcd_value = mcd_toolbox.calculate_mcd(os.path.join(t_gt_path, wav_name), os.path.join(t_output_path, wav_name))
        mcd += mcd_value
        mcd_count += 1
        if mcd_count % 100 == 0:
            print(mcd_count, mcd/mcd_count)

100 8.35187807190436
200 8.390435571753672
300 8.423236111250777
400 8.29415496798119
500 8.300448298817582
600 8.329507913179528
700 8.301066396498667
800 8.331559100039943
900 8.342863679419859
1000 8.31056878174379
1100 8.357093828898023
1200 8.423498355210173
1300 8.421160599658865
1400 8.443242400567577
1500 8.461479024561152
1600 8.46661786018398
1700 8.456371525342329
1800 8.4184887711868


In [37]:
print(mcd_count, mcd/mcd_count)

1834 8.416269685855791
